In [ ]:
import collections
import sklearn.metrics as metrics
import torch as th
import torch.nn.functional as F
import torch.nn.modules.loss as loss
import torch.optim as optim
import torch.utils as utils
import data
import my
import lenet
import resnet

In [ ]:
def global_scores(c, loader):
    keys = ('accuracy', 'precision', 'recall', 'f1')
    scores = (
        metrics.accuracy_score,
        lambda y, y_bar: metrics.precision_score(y, y_bar, average='micro'),
        lambda y, y_bar: metrics.recall_score(y, y_bar, average='micro'),
        lambda y, y_bar: metrics.f1_score(y, y_bar, average='micro'),
    )
    values = [value.item() for value in my.global_scores(c, loader, scores)]
    return collections.OrderedDict(zip(keys, values))

In [ ]:
train_x, train_y, test_x, test_y = data.load_cifar10(rbg=True, torch=True)

train_set = utils.data.TensorDataset(train_x, train_y)
train_loader = utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
test_set = utils.data.TensorDataset(test_x, test_y)
test_loader = utils.data.DataLoader(test_set, batch_size=4096)

n_classes = int(train_y.max() - train_y.min() + 1)

In [ ]:
c = lenet.LeNet(3, n_classes)
# c = lenet.LeNet(3, n_classes, nonlinear=F.relu)
# c = resnet.ResNet(18, n_classes)
cuda = True # always GPU 0
if cuda:
    c.cuda()
optimizer = optim.SGD(c.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4)
# optimizer = optim.Adam(c.parameters())

for key, value in global_scores(c, test_loader).items():
    print(key, value)

In [ ]:
n_epochs = 25
for i in range(n_epochs):
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        ce = loss.CrossEntropyLoss()(c(x), y)
        optimizer.zero_grad()
        ce.backward()
        optimizer.step()
    
    train_scores = global_scores(c, train_loader)
    test_scores = global_scores(c, test_loader)
    print('[epoch %d]' % (i + 1) + \
          ' | '.join('%s %0.3f/%0.3f' % (key, value, test_scores[key])for key, value in train_scores.items()))

In [ ]:
for pg in optimizer.param_groups:
    pg['lr'] *= 0.1

In [ ]:
# th.save(c.state_dict(), 'cifar10-resnet18')
# th.save(c.state_dict(), 'cifar10-1-9-resnet18')

# Results

## LeNet

## ResNet